Instructions:
1) In this final lab, we will model our data. Import sklearn train_test_split and separate the data.
2) Try a simple linear regression with all the data to see whether we are getting good results.
3) Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.
4) Use the function to check LinearRegressor and KNeighborsRegressor.
5) You can check also the MLPRegressor for this task!
6) Check and discuss the results.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


%matplotlib inline

pd.set_option('display.max_columns', None)

## Data collection and exploration

In [ ]:
customer_df = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

In [ ]:
customer_df.head()

In [ ]:
customer_df.shape

In [ ]:
customer_df.info()

In [ ]:
# Change capitalization to lowercase and replace spaces with underscores:
customer_df.columns = customer_df.columns.str.lower().str.replace(" ", "_")
customer_df.head()

In [ ]:
customer_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'])

In [ ]:
round((customer_df.isna().sum()/len(customer_df)*100),2)

In [ ]:
customer_df.customer.unique()

In [ ]:
numericals = customer_df.select_dtypes(np.number)

In [ ]:
numericals

In [ ]:
for column in numericals:
    print(numericals[column].unique())

In [ ]:
for column in numericals:
    numericals[column].hist()
    plt.title(column)
    plt.show()

In [ ]:
def check_dtypes(numericals):
    data = numericals.dtypes
    x=0
    for i in range (len(numericals.columns)):
        print(data[x]," ", numericals.iloc[0][numericals.columns[i]]," ",numericals.columns[i])
        x+=1
    return()

In [ ]:
check_dtypes(numericals)

In [ ]:
# Discrete: number_of_policies, number_of_open_complaints, months_since_policy_inception, months_since_last_claim
# Continuous: monthly_premium_auto, total_claim_amount,income,customer_lifetime_value
# Why monthly_premium_auto is in continous: it's calculated based 
# on a lot of different criteria (type of cars, policies taken, age of the main driver, how many accidents you had and when etc.)
# so with all these combinations of inputs we can get any number of monthly_premium_auto.

In [ ]:
numericals['income'] = numericals['income'].astype(float)
numericals['monthly_premium_auto'] = numericals['income'].astype(float)
check_dtypes(numericals)

In [ ]:
def to_dict(df):
    for col in df.columns:
        if df[col].dtypes == 'int64':
            df1 = df.select_dtypes(include='int64')
            discrete = df1.to_dict()
        else:
            df2 = df.select_dtypes(include='float64')
            continuous = df2.to_dict()
    return continuous, discrete

In [ ]:
continuous, discrete = to_dict(numericals)

In [ ]:
type(discrete)

In [ ]:
discrete.keys()

In [ ]:
continuous.keys()

In [ ]:
cat = customer_df.select_dtypes(np.object)
cat.head()

In [ ]:
for column in cat:
    print(cat[column].unique())

In [ ]:
for column in cat:
    print(len(cat[column].unique()))

In [ ]:
check_dtypes(cat)

In [ ]:
# customer => continuous but would be better to set it as the index
# all the other variables are discrete
# text in cat variable is already clean

In [ ]:
# not sure if we're meant to add the cat variables to the discrete variable
cat_dict = cat.to_dict()

In [ ]:
cat_dict.keys()

In [ ]:
final_discrete = discrete | cat_dict

In [ ]:
final_discrete.keys()

In [ ]:
discrete_df = pd.DataFrame.from_dict(discrete)

In [ ]:
discrete_df.head()

In [ ]:
discrete_df.reset_index(drop=True, inplace=True)
cat.reset_index(drop=True, inplace=True)
discrete_df = pd.concat([discrete_df, cat], axis=1)
discrete_df.shape

In [ ]:
discrete_df

In [ ]:
continuous_df = pd.DataFrame.from_dict(continuous)

In [ ]:
continuous_df.head()

In [ ]:
# 10. What should we do with the customer_id column? => should be set as the index
customer_df = customer_df.set_index('customer')
customer_df.head()

## Lab | Cleaning categorical data


3. Plot a correlation matrix, what can you see?

In [ ]:
#corr for continous_df
continuous_df.corr()

In [ ]:
mask = np.zeros_like(continuous_df.corr()) 
mask[np.triu_indices_from(mask)] = True 
#use mask to see just 50% of the matrix -> easay to "read"
fig, ax = plt.subplots(figsize=(12, 8))
ax = sns.heatmap(continuous_df.corr(), mask=mask, annot=True)
plt.show()

In [ ]:
# income and monthly_premium_auto have correlation of 1
# otherwise there are no special findings

In [ ]:
# corr for descrete_dfabs

In [ ]:
discrete_df.corr()

In [ ]:
mask = np.zeros_like(discrete_df.corr()) 
mask[np.triu_indices_from(mask)] = True 
#use mask to see just 50% of the matrix -> easay to "read"
fig, ax = plt.subplots(figsize=(12, 8))
ax = sns.heatmap(discrete_df.corr(), mask=mask, annot=True)
plt.show()

In [ ]:
# just low correlations 

In [ ]:
# Create a function to plot every discrete variables
for column in discrete_df:
    discrete_df[column].hist()
    plt.title(column)
    plt.show()

In [ ]:
# Create a function to plot every conitnous variables
#first include defien new variable with "total_claim_amount" to compare
cols_to_include = continuous_df.loc[:, continuous_df.columns != 'total_claim_amount']
for column in cols_to_include:
    plt.figure(figsize=(10,5))
    sns.scatterplot(continuous_df[column],continuous_df['total_claim_amount'])
    plt.show()

In [ ]:
# Observations: there seems to be not a lot of correlation 
# between the monthly_premium_auto, the income and the target
# But there is a correlation between the 
# customer_lifetime_value and the target

In [ ]:
#boxplot for continuous variables
for col in continuous_df:
    plt.figure(figsize=(10,5))
    sns.boxplot(x=continuous_df[col])
    plt.show()

In [ ]:
# Total_claim_amount and customer_lifetime_value have the most outliers

In [ ]:
def remove_outliers(df, threshold=1.5):
    numerical = df.select_dtypes(np.number)
    columns = numerical.columns
    for column in columns:
        if len(df[column].unique()) < 10:
            continue
        else:
            upper = np.percentile(df[column], 75)
            lower = np.percentile(df[column], 25)
            iqr = upper - lower
            upper_limit = upper + threshold * iqr
            lower_limit = lower - threshold * iqr
            df = df[(df[column]>lower_limit) & (df[column]<upper_limit)]
    return df

In [ ]:
continuous = remove_outliers(continuous_df, threshold=1.5)

In [ ]:
for col in continuous:
    plt.figure(figsize=(10,5))
    sns.boxplot(x=continuous[col])
    plt.show()

In [ ]:
# Checking NaNs per column
def check_nan(df):
    nulls = pd.DataFrame(df.isna().sum()/len(df))
    nulls= nulls.reset_index()
    nulls.columns = ['column_name', 'Percentage Null Values']
    nulls.sort_values(by='Percentage Null Values', ascending = False)
    return nulls

In [ ]:
cont = check_nan(continuous)
cont

In [ ]:
disc = check_nan(discrete_df)

In [ ]:
disc

In [ ]:
# 10. for the categorical data, check if there is some kind of text in a variable so we would need to clean it. 
# Hint: Use the same method you used in step 7. 
# Depending on the implementation, decide what to do with the variables you get.
cat.head()

In [ ]:
for col in cat:
    x = cat[col].unique()
    print(x)

In [ ]:
# 11 to 12 is already done

# lab-feature-extraction

In [ ]:
# 1. Open the categoricals variable we created before.

In [ ]:
cat.head()

In [ ]:
#2. Plot all the categorical variables with the proper plot. What can you see?

In [ ]:
# use histograms to see distribution

In [ ]:
customer_df.head()

In [ ]:
customer_df.head

In [ ]:
cat.head()

In [ ]:
customer_df['total_claim_amount']

In [ ]:
continuous_df['total_claim_amount']

In [ ]:
customer_df['total_claim_amount']

In [ ]:
# use boxplot to see outliers
# also here: compare
for col in cat:
    plt.figure(figsize=(8,5))
    sns.boxplot(x=cat[col], y=continuous_df['total_claim_amount'])

In [ ]:
# use also barplot to see relationship between a numeric and a categoric variable
# for numeric choose target "total_claim_amount"

In [ ]:
for col in cat:
    plt.figure(figsize=(8,5))
    sns.barplot(x=cat[col], y=continuous_df['total_claim_amount'])

In [ ]:
# 3. There might be some columns that seem to be redundant, check their values to be sure. What should we do with them?
# gender and sales_channel
# also policy_type and vehicle_size

In [ ]:
cat = cat.drop(['gender','policy_type','sales_channel', 'vehicle_size'], axis=1)

In [ ]:
cat.head()

In [ ]:
# 4. Plot time variable. Can you extract something from it?

In [ ]:
# try barplot -> makes no sense
plt.figure(figsize=(8,5))
sns.barplot(x=customer_df['effective_to_date'], y=customer_df['total_claim_amount'])
plt.show()

In [ ]:
# use sns lineplot
fig, ax = plt.subplots(figsize=(15, 6))
sns.lineplot(customer_df['effective_to_date'], customer_df['total_claim_amount'] )

# Lab | Data cleaning and wrangling


1. We will start with removing outliers. So far, we have discussed different methods to remove outliers.
Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

In [ ]:
def remove_outliers(df, threshold=1.5):
    numerical = df.select_dtypes(np.number)
    columns = numerical.columns
    for column in columns:
        if len(df[column].unique()) < 10:
            continue
        else:
            upper = np.percentile(df[column], 75)
            lower = np.percentile(df[column], 25)
            iqr = upper - lower
            upper_limit = upper + threshold * iqr
            lower_limit = lower - threshold * iqr
            df = df[(df[column]>lower_limit) & (df[column]<upper_limit)]
    return df

2. Create a copy of the dataframe

In [ ]:
data_copy = customer_df.copy()

3. Normalize the continuous variables. You can use any one method you want.

In [ ]:
# using standardscale
transformer = StandardScaler().fit(continuous)
x_standardized = transformer.transform(continuous)
X = pd.DataFrame(x_standardized)
X.columns = continuous.columns
X.head()

In [ ]:
#Merging the numerical dataframes:
len(discrete_df)-len(X)

In [ ]:
# do a left join
X.reset_index(drop=True, inplace=True)
discrete_df.reset_index(drop=True, inplace=True)
numerical_df = pd.merge(X, discrete_df, how='left', left_index=True, right_index=True)
numerical_df

In [ ]:
# 4. Encode the categorical variables
cat_encoded = pd.get_dummies(cat, columns=cat.columns, drop_first=True)
cat_encoded.head()

In [ ]:
cat_encoded.shape

5. The time variable can be useful. Try to transform its data into a useful one.
Hint: Day week and month as integers might be useful. \ => Has already been done in order to plot the datetime data. \ I will apply the same method on data_copy and drop the effective_to_date column

In [ ]:
data_copy['month']= data_copy['effective_to_date'].dt.month
data_copy['day']= data_copy['effective_to_date'].dt.day
data_copy = data_copy.drop(['effective_to_date'], axis=1)

In [ ]:
data_copy.head()

6. Since the model will only accept numerical data:
Check and make sure that every column is numerical, if some are not, change it using encoding: \ \ One hot to state \ Ordinal to coverage \ Ordinal to employmentstatus \ Ordinal to location code \ One hot to marital status \ One hot to policy type \ One hot to policy \ One hot to renew offercustomer_df \ One hot to sales channel \ One hot vehicle class \ Ordinal vehicle size \ \ data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2}) \ given that column "coverage" in the dataframe "data" has three categories: \ "basic", "extended", and "premium" and values are to be represented in the same order.

In [ ]:
#One hot encoder
encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(data_copy[['state', 'marital_status','policy_type','policy','renew_offer_type', 'sales_channel','vehicle_class']])
encoded = encoder.transform(data_copy[['state', 'marital_status','policy_type','policy','renew_offer_type', 'sales_channel','vehicle_class']]).toarray()
encoded

In [ ]:
encoded_cat = pd.DataFrame(encoded)
encoded_cat

In [ ]:
encoded_cat.columns = encoder.get_feature_names()

In [ ]:
encoded_cat

In [ ]:
# ORDINAL ENCODING
# Ordinal to coverage \ Ordinal to employmentstatus \ Ordinal to location code \ Ordinal vehicle size
ordinal = data_copy[['coverage','employmentstatus','location_code','vehicle_size']]
ordinal

In [ ]:
# 1. encode the coverage

In [ ]:
# see unique values
ordinal.coverage.value_counts()

In [ ]:
# Intuitively we want to rank like this:
# Define a dictionary for encoding variable
cov_dict = {'Basic':0,
            'Extended':1,
            'Premium':2}

In [ ]:
# Create the mapped values in a new column
ordinal['coverage'] = ordinal['coverage'].map(cov_dict)

In [ ]:
# Check the data
ordinal

In [ ]:
# 2. Encode the employmentstatus

In [ ]:
# first we'll put Medical leave, Disabled and Retired in a new category called 'other'.
# we could also put these in unemployed but I'd say from a monetary perspective they wouldn't get the same revenues,
# for ex, a retired person has (ideally) more revenue and proprietary goods than an unemployed person
count_emp = pd.DataFrame(ordinal['employmentstatus'].value_counts())
count_emp = count_emp.reset_index()
count_emp.columns = ['status', 'counts']
other_df = count_emp[count_emp['counts']<500]
other_df = list(other_df['status'])
other_df

In [ ]:
# function to get the three categories "employed, unemployed, other"
def clean_employement_status(x):
    if x in other_df:
        return 'Other'
    else:
        return x

ordinal['employmentstatus'] = list(map(clean_employement_status, ordinal['employmentstatus']))
ordinal.employmentstatus.value_counts()

In [ ]:
# Now how would we order it, to better serve our analysis and in a way that is relevant to our target?
plt.figure(figsize=(8,5))
sns.barplot(x=ordinal['employmentstatus'],y=customer_df['total_claim_amount'], color='turquoise')

In [ ]:
# The 'least' employed have the most claim amount - I will order it from 0 (least employment) to 2 (most employment)
# Define a dictionary for encoding variable
emp_dict = {'Unemployed':0,
            'Other':1,
            'Employed':2}

In [ ]:
# Create the mapped values in a new column
ordinal['employmentstatus'] = ordinal['employmentstatus'].map(emp_dict)

In [ ]:
# 3. Encode the location_code

In [ ]:
# we can see the more cars, the higher the total_claim_amount
plt.figure(figsize=(8,5))
sns.barplot(x=ordinal['location_code'],y=customer_df['total_claim_amount'], color='turquoise')

In [ ]:
# Define a dictionary for encoding variable
loc_dict = {'Rural':0,
            'Urban':1,
            'Suburban':2}

In [ ]:
# Create the mapped values in a new column
ordinal['location_code'] = ordinal['location_code'].map(loc_dict)

In [ ]:
# 4. Encode the vehicle_size

In [ ]:
# Define a dictionary for encoding variable
car_dict = {'Small':0,
            'Medsize':1,
            'Large':2}


In [ ]:
# Create the mapped values in a new column
ordinal['vehicle_size'] = ordinal['vehicle_size'].map(car_dict)

In [ ]:
# concatinate all encoded categorical data

In [ ]:
ordinal.reset_index(drop=True, inplace=True)
encoded_cat.reset_index(drop=True, inplace=True)
final_cat = pd.concat([ordinal, encoded_cat], axis=1)
final_cat

In [ ]:
# merge all data (final_cat and numerical_df) together 
data = pd.merge(numerical_df, final_cat, how='left', left_index=True, right_index=True)
data = data.drop('total_claim_amount', axis=1)
target = pd.DataFrame(customer_df['total_claim_amount'])
data.reset_index(drop=True, inplace=True)
target.reset_index(drop=True, inplace=True)
data = pd.merge(data, target, how='left', left_index=True, right_index=True)
data

In [ ]:
# check for NaNs
data.isna().sum()

# Lab | Comparing regression models


In [ ]:
# In this final lab, we will model our data. Import sklearn train_test_split and separate the data.

In [ ]:
# seperate target
y = data['total_claim_amount']
X = data.drop('total_claim_amount', axis = 1)

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=42)

## Linear Regression


In [ ]:
def modeling_1(y, X, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train )
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    RMSE = mean_squared_error(y_test, predictions, squared=False)
    MSE = mean_squared_error(y_test, predictions)
    MAE = mean_absolute_error(y_test, predictions)
    print("R2 =", round(r2,2)), print("RMSE =", round(RMSE,2)), print("MSE =", round(MSE,2)), print("MAE =", round(MAE,2))
    return predictions, y_test, r2

In [ ]:
predictions1, y_test1, r2_1 = modeling_1(y, X, test_size=0.24)

3. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

In [ ]:
def modeling(y, X, models=[], test_size=0.3):
    for model in models:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        model.fit(X_train, y_train )
        predictions = model.predict(X_test)
        r2 = r2_score(y_test, predictions)
        r2_adj = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
        RMSE = mean_squared_error(y_test, predictions, squared=False)
        MSE = mean_squared_error(y_test, predictions)
        MAE = mean_absolute_error(y_test, predictions)
        print(model, 'metrics are: '), print("R2 =", round(r2,2)), print("R2 adjusted =", round(r2_adj,2)), print("RMSE =", round(RMSE,2)), print("MSE =", round(MSE,2)), print("MAE =", round(MAE,2))
    return predictions, y_test, r2

In [ ]:
# Use the function to check LinearRegressor and KNeighborsRegressor.

In [ ]:
predictions, y_test, r2 = modeling(y, X, models=[LinearRegression(), KNeighborsRegressor(n_neighbors=10)], test_size=0.24)

4. You can check also the MLPRegressor for this task!

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

In [ ]:
predictions, y_test, r2 = modeling(y, X, models=[LinearRegression(), KNeighborsRegressor(n_neighbors=10), MLPRegressor(max_iter=100)], test_size=0.24)

5. Check and discuss the results.

Let's check how the predictions compare to the results. \ We could improve the function to run different k values and return the best predictions. \ Same with the MLP regressor and the max_iter values.

## Finding the best K value


In [ ]:
scores = []
for i in range(2,15): 
    model = KNeighborsRegressor(n_neighbors=i)
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(2,15), scores, color = 'blue', linestyle='dashed',
marker='o', markerfacecolor='red', markersize=10)
plt.title('R2 vs. K Value')
plt.xlabel('K')
plt.ylabel('R2')

In [ ]:
# -> the best k-value is 10 

In [ ]:
k_predictions, k_y_test, k_r2 = modeling(y, X, models=[KNeighborsRegressor(n_neighbors=10)], test_size=0.24)

In [ ]:
LR_predictions, LR_y_test, LR_r2 = modeling(y, X, models=[LinearRegression()],test_size=0.24)

In [ ]:
MLP_predictions, y_test, MLP_r2 = modeling(y, X, models=[MLPRegressor(max_iter=250)],test_size=0.24)

In [ ]:
k_results = diff_df(y_test, k_predictions)
k_results

In [ ]:
LR_results = diff_df(y_test, LR_predictions)
LR_results

In [ ]:
MLP_results = diff_df(y_test, MLP_predictions)
MLP_results

In [ ]:
def we_like_to_see(results):
    beautiful_graph = sns.regplot(results['true'], results['pred'])
    return beautiful_graph

In [ ]:
k_beautiful_graph = we_like_to_see(k_results)

In [ ]:
LR_beautiful_graph = we_like_to_see(LR_results)

In [ ]:
LR_beautiful_graph = we_like_to_see(LR_results)

In [ ]:
# The most accurate results are obtained with the MLP regressor.

# The models could possibly be better at predicting the target, we can try revisiting the data

In [ ]:
mask = np.zeros_like(numerical_df.corr())
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(12, 8))
ax = sns.heatmap(numerical_df.corr(), mask=mask, annot=True)
plt.show()

In [ ]:
mask = np.zeros_like(continuous.corr())
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.heatmap(continuous.corr(), mask=mask, annot=True)
plt.show()

In [ ]:
mask = np.zeros_like(discrete_df.corr())
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.heatmap(discrete_df.corr(), mask=mask, annot=True)
plt.show()

In [ ]:
mask = np.zeros_like(ordinal.corr())
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.heatmap(ordinal.corr(), mask=mask, annot=True)
plt.show()

In [ ]:
# Very little correlation between the discrete variables and the target, so let's run the model without those.

In [ ]:
no_discrete = data.drop(columns=discrete_df.columns, axis=1)
no_discrete

In [ ]:
no_dis_y = no_discrete['total_claim_amount']
no_dis_X = no_discrete.drop('total_claim_amount', axis=1)
no_dis_pred, no_dis_y_test, no_dis_r2 = modeling(no_dis_y, no_dis_X, models=[LinearRegression(), KNeighborsRegressor(n_neighbors=6), MLPRegressor(max_iter=250)], test_size=0.24)

Differences:
The KNN model seems to perform better once we remove the variables that are not statistically significant. We can recalculate the better k values and the difference with the test data.

In [ ]:
no_dis_X_train, no_dis_X_test, no_dis_y_train, no_dis_y_test = train_test_split(no_dis_X, no_dis_y, test_size=0.24, random_state=42)

In [ ]:
scores = []
for i in range(2,15): 
    model = KNeighborsRegressor(n_neighbors=i)
    model.fit(no_dis_X_train, no_dis_y_train)
    scores.append(model.score(no_dis_X_test, no_dis_y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(2,15), scores, color = 'blue', linestyle='dashed',
marker='o', markerfacecolor='red', markersize=10)
plt.title('R2 vs. K Value')
plt.xlabel('K')
plt.ylabel('R2')

In [ ]:
# -> the best k-value is 6 

In [ ]:
k2_predictions, k2_y_test, k2_r2 = modeling(no_dis_y, no_dis_X, models=[KNeighborsRegressor(n_neighbors=6)], test_size=0.24)

In [ ]:
k2_results = diff_df(no_dis_y_test, k2_predictions)
k2_results

In [ ]:
k2_beautiful_graph = we_like_to_see(k2_results)

In [ ]:
LR_beautiful_graph = we_like_to_see(LR_results)

In [ ]:
MLP_beautiful_graph = we_like_to_see(MLP_results)

In [ ]:
# FAZIT: Even though the KNN performs better now, it's still not as accurate as the other two.